In [1]:
from src.model import Config,GPT
from src.inference import GPTInfer,HNSWRetriever
import tiktoken
import torch
from sentence_transformers import SentenceTransformer


c:\Users\chris\.conda\envs\deep_learning_cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [3]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
retriever = HNSWRetriever()

In [4]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'    # for apple macbook GPUs
print(f'using device: {device}')

using device: cuda


In [5]:
model_path = './model_02399.pt'
checkpoint = torch.load(model_path, weights_only=False)
model = GPT(config=checkpoint['config'])
model.load_state_dict(checkpoint['model'])
model = model.to(device)
token_encoder = tiktoken.get_encoding('gpt2')
generator = GPTInfer(model, token_encoder, device,retriever,embed_model)

In [6]:
prompt = "Once upon a time"

In [7]:
tokens = generator.token_encoder.encode(prompt)
prev_tok = len(tokens)

In [ ]:
prompt = generator.print_stream(
    prompt,
    max_new_tokens=1024,
    seed=42,
    temperature=0.8,        
    top_p=0.9,              
    top_k=None,             
    repetition_penalty=1.2, 
    frequency_penalty=0.6,  
    no_repeat_ngram_size=3, 
    longer_story=True,
    rag_k=3,
)
tokens = generator.token_encoder.encode(prompt)
prev_tok = len(tokens)

Once upon a time , I could not help noticing . At last we saw the iron bar near me , and looked about me at the place where I was standing , and when my eyes turned to mine he looked up from his seat again with an expression of wonder . He had good reason for coming out of sight ; but as we went away he let go on , saying that it was my fathers habitation for a long while since—when I returned home and lived in the house as soon as possible , lest he should be there in peace at once—by-and-by ; and then all four entered into conversation with each other by himself ; they talked much together of their conversation ; such talk about life being called the devil with him !  or howl as if our mutual interest were still worth listening to them without fear . The very next day he took up his abode amongst us : every night after day passed on through black passages between rain-clapped women who knew nothing more than were necessary (to use this womand husband) together ; it seemed like seeing